# EHV Term to Graphite

Generate exports for a curated list of possible canddiate terms to be added to the EHV, using extractions from HAWC.

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
%env DJANGO_ALLOW_ASYNC_UNSAFE true
%env DJANGO_SETTINGS_MODULE hawc.main.settings.dev

env: DJANGO_ALLOW_ASYNC_UNSAFE=true
env: DJANGO_SETTINGS_MODULE=hawc.main.settings.dev


In [3]:
import django
django.setup()

In [4]:
from datetime import datetime
import json

import pandas as pd
from django.db.models import Q

from hawc.apps.animal.models import Endpoint

today = datetime.now().strftime('%Y-%m-%d')

### SR Automation Imports

In [5]:
import spacy
from spacy.matcher import PhraseMatcher

import pandas as pd
import numpy as np
import re
import string
from rapidfuzz import fuzz, process

In [6]:
nlp = spacy.load("en_core_web_sm")

## Get all endpoints where EHV is unused
### Requires a local copy of a HAWC deployment's database

Preserve endpoint relations. This is designed to give a little more metadata for assistance in mapping relations, but is not as useful for ranking or prioritization.

In [7]:
def to_df(qs):
    data=qs.values_list(
        'id', 'animal_group__experiment__study__short_citation', 'assessment__vocabulary', 
        'system', 'organ', 'effect', 'effect_subtype', 'name',
        'system_term_id', 'organ_term_id', 'effect_term_id', 'effect_subtype_term_id', 'name_term_id',
        'created', 'last_updated'
    )
    df =  pd.DataFrame(
        data=data, 
        columns=[
            'endpoint_id', 'study_citation', 'vocabulary', 
            'system', 'organ', 'effect', 'effect_subtype', 'name',
            'system_term_id', 'organ_term_id', 'effect_term_id', 'effect_subtype_term_id', 'name_term_id',
            'created', 'last_updated'
            
        ]
    )
    df.loc[:, "vocabulary"] = df.vocabulary.fillna(0)
    df.loc[:, "vocabulary"] = df.vocabulary.map({0: False, 1: True})
    df[['system_term_id', 'organ_term_id', 'effect_term_id', 'effect_subtype_term_id', 'name_term_id']] = \
        df[['system_term_id', 'organ_term_id', 'effect_term_id', 'effect_subtype_term_id', 'name_term_id']].fillna("NULL")
    return df


q_filters = (
    Q(system_term_id__isnull=True) | 
    Q(organ_term_id__isnull=True) | 
    Q(effect_term_id__isnull=True) | 
    Q(effect_subtype_term_id__isnull=True) | 
    Q(name_term_id__isnull=True)
)

df1 = to_df(Endpoint.objects.filter(q_filters).order_by('-last_updated').select_related('assessment', 'animal_group__experiment__study'))
df1.head()

,endpoint_id,study_citation,vocabulary,system,organ,effect,effect_subtype,name,system_term_id,organ_term_id,effect_term_id,effect_subtype_term_id,name_term_id,created,last_updated
0,100580551,Witchey SK et al. 2023,True,Nervous,"Brain Weight, Cholinesterase",,,Nervous Effects,NULL,NULL,NULL,NULL,NULL,2024-10-01 04:41:27.211868+00:00,2024-10-01 04:41:27.211880+00:00
1,100580543,Witchey SK et al. 2023,True,Developmental,"Offspring Body Weight, Vaginal Opening,",,,Developmental Effects,NULL,NULL,NULL,NULL,NULL,2024-10-01 03:48:10.954082+00:00,2024-10-01 04:37:04.520624+00:00
2,100580542,Witchey SK et al. 2023,True,Developmental,"Offspring Body Weight, Balanopreputial Seperat...",,,Developmental Effects,NULL,NULL,NULL,NULL,NULL,2024-10-01 03:43:08.505855+00:00,2024-10-01 04:36:31.503252+00:00
3,100580544,Witchey SK et al. 2023,True,Hepatic,"Liver Weight, Cholinesterase",,,Hepatic Effects,NULL,NULL,NULL,NULL,NULL,2024-10-01 04:01:31.441228+00:00,2024-10-01 04:34:59.110171+00:00
4,100580550,Witchey SK et al. 2023,True,Hepatic,Cholinesterase,,,Hepatic Effects,NULL,NULL,NULL,NULL,NULL,2024-10-01 04:33:14.118446+00:00,2024-10-01 04:34:13.466542+00:00


## Get most frequently used missing terms

Filter for cases where EHV is enabled, values count where term is repeated and missing. Filter to only include cases where the same term was used more than once.

In [8]:
from django.db.models import Count, F

In [9]:
qs1 = (
    Endpoint.objects
        .filter(assessment__vocabulary=1, system_term_id__isnull=True)
        .values("system")
        .annotate(count=Count('id', distinct=True))
        .order_by('-count')
        .filter(count__gt=2)
)
qs2 = (
    Endpoint.objects
        .filter(assessment__vocabulary=1, organ_term_id__isnull=True)
        .values("organ")
        .annotate(count=Count('id', distinct=True))
        .order_by('-count')
        .filter(count__gt=2)
)
qs3 = (
    Endpoint.objects
        .filter(assessment__vocabulary=1, effect_term_id__isnull=True)
        .values("effect")
        .annotate(count=Count('id', distinct=True))
        .order_by('-count')
        .filter(count__gt=2)
)
qs4 = (
    Endpoint.objects
        .filter(assessment__vocabulary=1, effect_subtype_term_id__isnull=True)
        .values("effect_subtype")
        .annotate(count=Count('id', distinct=True))
        .order_by('-count')
        .filter(count__gt=2)
)
qs5 = (
    Endpoint.objects
        .filter(assessment__vocabulary=1, name_term_id__isnull=True)
        .values("name")
        .annotate(count=Count('id', distinct=True))
        .order_by('-count')
        .filter(count__gt=2)
)

df2 = pd.concat([
       pd.DataFrame(qs1).assign(type="system").rename(columns={"system":"term"}),
       pd.DataFrame(qs2).assign(type="organ").rename(columns={"organ":"term"}),
       pd.DataFrame(qs3).assign(type="effect").rename(columns={"effect":"term"}),
       pd.DataFrame(qs4).assign(type="effect_subtype").rename(columns={"effect_subtype":"term"}),
       pd.DataFrame(qs5).assign(type="name").rename(columns={"name":"term"}),
]).sort_values('count', ascending=False)[['type', 'term', 'count']]
print(df2.shape[0])
df2.head(10)

2381


,type,term,count
0,system,Developmental,3456
0,organ,Whole Body,3351
0,effect,Pregnancy Outcome,2931
0,effect_subtype,Clinical Observation,2089
1,effect_subtype,Histopathology,1949
1,system,Nervous,1687
1,effect,Behavioral Function,1610
2,effect_subtype,Offspring Growth,1582
1,organ,CNS,1466
2,effect,Histopathology,1360


## Write exports

In [10]:
df1.to_csv(f'data/hawc-candidate-terms-endpoints.csv', index=False)
df2.to_csv(f'data/hawc-candidate-terms-ranked.csv', index=False)

## Identify EHV candidate terms for manual curation or synonyms to incorporate

### Read

In [11]:
hawc = pd.read_csv(r'data/hawc-candidate-terms-endpoints.csv')
ehv  = pd.read_excel(r'data/ehv.xlsx')
OUTPUT_FOLDER = r'data/Output'

### String Cleaning Functions

In [12]:
def removeCodes(text):  # removes numeric codes at the end of a string (that start with XXX or are numbers)
    text = re.sub(r'XXX\d+$', '', str(text)).strip()
    text = re.sub(r'\d+$', '', str(text)).strip()
    return(text)

def remove_characters(text, character_list = ["?","!",".","/"] ): # replaces punctuation with space + punctuation + space 
    text = text.translate(''.join(character_list))
    return(text)

def replace_dashes(text): # replaces dashes with spaces
    text = text.replace('-',' ')
    return(text)

def replace_surpluswhitespace(text): # replace multiple whitespaces with single space and gets rid of trailing and leading spaces.
    text =' '.join(text.split())
    return(text)

def lowerText(text):
    text = text.lower()
    return(text)

def removeParentheses(text):
    text = re.sub(r"[\(\[].*?[\)\]]", "", str(text))
    return(text)

def lemmaString(text):
    doc = nlp(text)
    text = [token.lemma_ for token in doc]
    text = ' '.join(text)
    return(text)

def preProcessing(text):
    text = removeParentheses(text)
    text = removeCodes(text)
    text = remove_characters(text)
    text = replace_dashes(text)
    text = replace_surpluswhitespace(text)
    text = lowerText(text)
    text = lemmaString(text)
    return(text)

In [13]:
hawc = hawc.iloc[:1000].copy() # make subset of dataset for quicker development iterations

In [14]:
%%timeit -n1 -r1
hawc["1_cleaned"] = hawc['name'].apply(preProcessing)

8.54 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit -n1 -r1
ehv['1_cleaned']  = ehv['name'].apply(preProcessing)

30.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [24]:
# hawc_ehv_merged = pd.merge(hawc, ehv, how='left', on="1_cleaned")

#### Match a candidate term from HAWC to an EHV term using Levenshtein Distance
##### This method also compares whether effect and effect_subtype match. Some entries in HAWC invert effect and effect_subtype. This method will resolve to "True" regardless of an inversion.

In [58]:
threshold = 99.99 # can add a threshold cutoff here and limit response output if desired

response = []
for i,row in hawc.iterrows():
    candidate_term = row["1_cleaned"]
    resp_match =  process.extractOne(candidate_term,ehv["1_cleaned"] , scorer=fuzz.token_sort_ratio)
    resp = row.to_dict()
    resp["match_score"] = resp_match[1] # add the Levenshtein Distance
    resp["join_on"] = resp_match[2]
    resp["effects_match"] = set([resp["effect"],resp["effect_subtype"]]).issubset( 
        set([ehv.iloc[resp_match[2]]["effect"],ehv.iloc[resp_match[2]]["effect_subtype"]])) # determine whether the effect and effect_types match
    response.append(resp)

results = pd.DataFrame(response)
results_merged = results.merge(ehv, right_index=True, left_on="join_on",suffixes=["_hawc","_ehv"])

In [59]:
results_merged.to_excel("data/FuzzyMatching_ExcludingParentheses_Lemma_Results.xlsx",index=False)